In [1]:
# Run this cell!
from datascience import *
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import requests

# United Healthcare Test Notebook
Tester notebook to pull data from United Healthcare Provider Network API and parse it into a Pandas DataFrame.

In [2]:
base_url = 'https://public.fhir.flex.optum.com/R4/'
practitioner_role = 'PractitionerRole?'
search_param = '_id=b60e69f615c24dfb1034f8c28ed68428c1306670f028880605dadff791b610c7'
full_query = base_url + practitioner_role + search_param

In [5]:
request = requests.get(full_query)
json_object = request.json()
json_object.keys()

dict_keys(['resourceType', 'id', 'meta', 'type', 'total', 'link', 'entry'])

In [23]:
entry = json_object.get('entry')[0] \
                   .get('resource')


{'reference': 'Organization/e67ec4a907a7b3a1a4bb3243a1436d9eaa509e697e4ec835c18005a05304f135'}

In [25]:
entry.keys()

dict_keys(['resourceType', 'id', 'meta', 'extension', 'active', 'practitioner', 'organization', 'specialty', 'location', 'healthcareService', 'telecom', 'availableTime'])

In [34]:
entry.get('extension')[-234]

{'url': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/StructureDefinition/network-reference',
 'valueReference': {'reference': 'Organization/cf4aa5140e63f08e8c0b4ec625fb62f55799feebb9ac2e755689285b6e515b6f'}}

In [24]:
entry

{'resourceType': 'PractitionerRole',
 'id': 'b60e69f615c24dfb1034f8c28ed68428c1306670f028880605dadff791b610c7',
 'meta': {'profile': ['http://hl7.org/fhir/us/davinci-pdex-plan-net/StructureDefinition/plannet-PractitionerRole'],
  'tag': [{'system': 'http://optum.com/fhir/wm/system/2021/07/resourceid',
    'code': '/dOo2IF+EbKnMgKq7GZmTE98NJDv6SYfkj+qvkaKyuQoj3/7yLVzKxcDLZYi2rHeuGa/LZtqUxw/F36etKivRL3gzFpefeVyCd3DjLWhl3/NAwXX1LDsemRFpDrHyKxJrG1npuh38hwVMp0/iud5'}]},
 'extension': [{'url': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/StructureDefinition/newpatients',
   'extension': [{'url': 'acceptingPatients',
     'valueCodeableConcept': {'coding': [{'system': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/CodeSystem/AcceptingPatientsCS',
        'code': 'newpt',
        'display': 'Accepting'}],
      'text': 'Accepting patients'}}]},
  {'url': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/StructureDefinition/network-reference',
   'valueReference': {'reference': 'Organization/f020b0

In [67]:
next_url_9510 = requests.get('https://public.fhir.flex.optum.com/R4?_getpages=3639863d-459f-489b-a4d8-9d6e851802ef&_getpagesoffset=9500&_bundletype=searchset&_pagetype=bf775e8a-6ef2-44bd-87a3-b7d801d6ea04').json()#['link'][1].get('url')
next_url_9510

{'resourceType': 'Bundle',
 'id': '8b50e79b48b44c47a852d3c3995656e3',
 'meta': {'lastUpdated': '2022-11-17T09:44:32.336+00:00'},
 'type': 'searchset',
 'total': 10000,
 'link': [{'relation': 'self',
   'url': 'https://public.fhir.flex.optum.com/R4?_getpages=3639863d-459f-489b-a4d8-9d6e851802ef&_getpagesoffset=9500&_bundletype=searchset&_pagetype=bf775e8a-6ef2-44bd-87a3-b7d801d6ea04'},
  {'relation': 'next',
   'url': 'https://public.fhir.flex.optum.com/R4?_getpages=3639863d-459f-489b-a4d8-9d6e851802ef&_getpagesoffset=9510&_count=10&_bundletype=searchset&_pagetype=bf775e8a-6ef2-44bd-87a3-b7d801d6ea04'},
  {'relation': 'previous',
   'url': 'https://public.fhir.flex.optum.com/R4?_getpages=3639863d-459f-489b-a4d8-9d6e851802ef&_getpagesoffset=9490&_count=10&_bundletype=searchset&_pagetype=bf775e8a-6ef2-44bd-87a3-b7d801d6ea04'}],
 'entry': [{'fullUrl': 'https://public.fhir.flex.optum.com/R4/Location/cbad026704326303e3e2e28ca9ac38e68df8ea2e0e2c3edd94ad7affc1344c9d',
   'resource': {'resource

In [65]:
requests.get('https://public.fhir.flex.optum.com/R4?_getpages=3639863d-459f-489b-a4d8-9d6e851802ef&_getpagesoffset=9507&_count=2&_bundletype=searchset&_pagetype=bf775e8a-6ef2-44bd-87a3-b7d801d6ea04').json()

{'resourceType': 'OperationOutcome',
 'issue': [{'severity': 'information',
   'code': 'processing',
   'diagnostics': 'Narrow down search criteria'},
  {'severity': 'information',
   'code': 'processing',
   'diagnostics': 'TrackingUuid = fd3d8605b8858f15d15cbad56e84469b'}]}

In [30]:
# Don't actually run this cell!
base_url = 'https://public.fhir.flex.optum.com/R4'
location_CA = '/Location?_count=100&address-state=CA'
full_url = base_url + location_CA
request = requests.get(full_url)
json_object = request.json()

uhc_providers = pd.json_normalize(json_object.get('entry'))
print('Initial Request Successful! Hang tight this will take quite some time...')
page_number = 1

#while True:
#    try:
#        if page_number % 100 == 0:
#            print(f'Completed: {page_number}')
#        next_url = json_object.get('link')[1].get('url')
#        next_request = requests.get(next_url)
#        next_json = next_request.json()
#        uhc_providers = pd.concat([uhc_providers, pd.json_normalize(next_json.get('entry'))])
#        page_number += 1
#    except:
#        print('Done!')
#        break

Initial Request Successful! Hang tight this will take quite some time...


## Data Wrangling

In [ ]:
# Assortment of user-defined data wrangling functions
def beautify_number(number):
    '''
    Helper function that takes a string of length 10 (standard
    length of US phone numbers) and converts it to standard
    US phone number format: (###) ###-####
    -----
    Input:
    
    number (str) - String of length 10 to convert
    -----
    Output:
    
    beautified_number (str) - Converted string
    '''
    area_code = number[:3]
    next_three = number[3:6]
    last_four = number[6:]
    beautified_number = f'({area_code}) {next_three}-{last_four}'
    return beautified_number

def extract_phone_fax(dictionary):
    '''
    Data wrangling/conversion helper function that takes a list of
    Python dictionaries (named dictionary in the str_to_dict 
    function) and creates a two-column DataFrame df with a 
    column for Phone number and Fax number (None if not provided)
    -----
    Input:
    
    dictionary (List) - Python List of dictionaries, each item in
                        dictionary will be of length 1 (if just 
                        phone is available) or length 2 (if both
                        phone and fax is available)
    -----
    Output:
    
    df (DataFrame) - Pandas DataFrame object with two columns:
                        **Phone** | **Fax**
                             x    |    x
                             x    |   None
                                 ...
    '''
    # Code is repeated for the sake of readability/interpretability!
    phone = []
    fax = []
    for i in np.arange(0, len(dictionary)):
        item = dictionary[i]
        # Only phone number available
        if len(item) == 1:
            phone_dict = item[0]
            phone_number_raw = phone_dict.get('value')
            phone_number = beautify_number(phone_number_raw) # Call helper
            phone.append(phone_number)
            fax.append(None)
        # If both phone number and fax is available
        else:
            phone_dict = item[0]
            fax_dict = item[1]
            phone_number_raw = phone_dict.get('value')
            fax_number_raw = fax_dict.get('value')
            phone_number = beautify_number(phone_number_raw) # Call helper
            fax_number = beautify_number(fax_number_raw) # Call helper
            phone.append(phone_number)
            fax.append(fax_number)
    df = pd.DataFrame(data={'Phone Number':phone, 'Fax Number': fax})
    return df

def extract_contact_info(arr):
    '''
    Data wrangling/conversion function to take str objects parsed
    from Anthem API calls and creates a two-column DataFrame df with a 
    column for Phone number and Fax number (None if not provided)
    -----
    Input:
    
    arr (NumPy Array) - NumPy Array object of same length as the rows of
                        anthem DataFrame with each item in arr corresponding
                        to a string representation of the telecom dictionary
                        pulled from Anthem's database
    -----
    Intermediaries:
    
    dictionary (List) - Python List object containing Python dictionaries

    -----
    Output:
    df (DataFrame) - Pandas DataFrame object with two columns:
                        **Phone** | **Fax**
                             x    |    x
                             x    |   None
                                 ...
    '''
    dictionary = []
    for i in np.arange(0, len(arr)):
        string = arr[i]
        
        # Convert single quotes to double quotes for json.load() &
        # Remove [] to "delist-ify" strings
        dict_str = re.sub("'", '"', string[1:-1])

        # Extract all strings that form a dictionary from original string
        extracted = re.findall('\{[\w\s,:"]+\}', dict_str) # extracted is a list!

        # Create a nested list of Python dictionaries
        dictionary.append([json.loads(string) for string in extracted])

    # Create two-column DataFrame
    df = extract_phone_fax(dictionary) # Call helper function
    
    return df

In [2]:
uhc_raw = pd.read_csv('ca_uhc_providers.csv')
uhc_raw.head()

,fullUrl,resource.resourceType,resource.id,resource.meta.profile,resource.meta.tag,resource.extension,resource.status,resource.name,resource.address.use,resource.address.type,resource.address.text,resource.address.line,resource.address.city,resource.address.state,resource.address.postalCode,resource.address.country,resource.position.longitude,resource.position.latitude,resource.partOf.reference
0,https://public.fhir.flex.optum.com/R4/Location...,Location,f1685aef55513b81acfe9b4a477a0f6c4a9f9e7ab1896d...,['http://hl7.org/fhir/us/davinci-pdex-plan-net...,[{'system': 'http://optum.com/fhir/wm/system/2...,[{'url': 'http://hl7.org/fhir/us/davinci-pdex-...,active,2617 E Chapman Ave Ste 301-Orange-CA-92869-928...,work,physical,2617 E Chapman Ave Ste 301 Orange CA 92869-3234,['2617 E Chapman Ave Ste 301'],Orange,CA,92869-3234,USA,-117.825418,33.787957,Location/f1685aef55513b81acfe9b4a477a0f6c4a9f9...
1,https://public.fhir.flex.optum.com/R4/Location...,Location,e9df60af0696253246e93d58075411b43b182a53cfab56...,['http://hl7.org/fhir/us/davinci-pdex-plan-net...,[{'system': 'http://optum.com/fhir/wm/system/2...,NaN,active,1535 W Merced Ave Ste 300-West Covina-CA-91790...,work,physical,1535 W Merced Ave Ste 300 West Covina CA 91790...,['1535 W Merced Ave Ste 300'],West Covina,CA,91790-3404,USA,-117.948138,34.062999,Location/e9df60af0696253246e93d58075411b43b182...
2,https://public.fhir.flex.optum.com/R4/Location...,Location,31jw582G4no3xpYYwQjvgSE93oBpJMWfCITnIgR2tZP,['http://hl7.org/fhir/us/davinci-pdex-plan-net...,[{'system': 'http://optum.com/fhir/wm/system/2...,[{'url': 'http://hl7.org/fhir/us/davinci-pdex-...,active,1081 N Tustin Ave Ste 113-Anaheim-CA-92807-928...,work,physical,1081 N Tustin Ave Ste 113 Anaheim CA 92807-1737,['1081 N Tustin Ave Ste 113'],Anaheim,CA,92807-1737,USA,-117.837694,33.853963,Location/31jw582G4no3xpYYwQjvgSE93oBpJMWfCITnI...
3,https://public.fhir.flex.optum.com/R4/Location...,Location,865ab32f4151a119a7f2184210384c8fca6b7f878707ca...,['http://hl7.org/fhir/us/davinci-pdex-plan-net...,[{'system': 'http://optum.com/fhir/wm/system/2...,NaN,active,1014 San Juan Ave # 1-7-Exeter-CA-93221-932211312,work,physical,1014 San Juan Ave # 1-7 Exeter CA 93221-1312,['1014 San Juan Ave # 1-7'],Exeter,CA,93221-1312,USA,-119.131834,36.298771,Location/865ab32f4151a119a7f2184210384c8fca6b7...
4,https://public.fhir.flex.optum.com/R4/Location...,Location,8c5eafeba55b616a92fc9874d301bf0c47b9f52a451d53...,['http://hl7.org/fhir/us/davinci-pdex-plan-net...,[{'system': 'http://optum.com/fhir/wm/system/2...,[{'url': 'http://hl7.org/fhir/us/davinci-pdex-...,active,12835 Pointe Del Mar Way Ste 2-Del Mar-CA-9201...,work,physical,12835 Pointe Del Mar Way Ste 2 Del Mar CA 9201...,['12835 Pointe Del Mar Way Ste 2'],Del Mar,CA,92014-3846,USA,-117.243025,32.932596,Location/8c5eafeba55b616a92fc9874d301bf0c47b9f...


In [21]:
uhc_raw.iloc[0, -1]

'Location/f1685aef55513b81acfe9b4a477a0f6c4a9f9e7ab1896df5b4de61afadd5e462'

In [4]:
'https://public.fhir.flex.optum.com/R4/HealthcareService/f1685aef55513b81acfe9b4a477a0f6c4a9f9e7ab1896df5b4de61afadd5e462'

'https://public.fhir.flex.optum.com/R4/HealthcareService/f1685aef55513b81acfe9b4a477a0f6c4a9f9e7ab1896df5b4de61afadd5e462'

In [28]:
requests.get('https://public.fhir.flex.optum.com/R4/HealthcareService?service-category=prov&location.address-state=CA').json().get('entry')

[{'fullUrl': 'https://public.fhir.flex.optum.com/R4/HealthcareService/811a1ea3e8179d2add3ad5e28a8691dc51638cf4a4bea24c09fe65034f605419',
  'resource': {'resourceType': 'HealthcareService',
   'id': '811a1ea3e8179d2add3ad5e28a8691dc51638cf4a4bea24c09fe65034f605419',
   'meta': {'profile': ['http://hl7.org/fhir/us/davinci-pdex-plan-net/StructureDefinition/plannet-HealthcareService'],
    'tag': [{'system': 'http://optum.com/fhir/wm/system/2021/07/resourceid',
      'code': 'wljpKzj76/Uw+ns1pEgsQUO7KuF66Fyg6GKRKtOlpSiVa5pR8dq4DO2tWPCV2uEfxl4X+W4JSJFqmsLwhUS8Yj2dmoIYX0cB6rUJwrY/j/SM15wzxkUQ/IS/7MVBa7uy+Aoe3XH0AyauPgbZ5peH'}]},
   'active': True,
   'providedBy': {'reference': 'Organization/721c2c9b833f6e54e778538470a502c502571cba66b4c8f97c208f52854fa20b'},
   'category': [{'coding': [{'system': 'http://hl7.org/fhir/us/davinci-pdex-plan-net/CodeSystem/HealthcareServiceCategoryCS',
       'code': 'prov',
       'display': 'Medical Provider'}],
     'text': 'Health care services rendered by a